In [1]:
include("fid_utils.jl");

# Calibration:
First calculate how much population error is introduced by rotation angle error when driving a $\pi$ pulse. We'll use this to convert $\pi$ pulse population error into variance of rotation angle.

In [2]:
function fid_π(εs)
    sw = SysWrapper(sys2, εs)
    ψ = R2_ratio(sw, 1, 2, 1) * sys2.ψ2s[1]
    return norm(sys2.proj2s[2, 2] * ψ)^2
end
display(fid_π([0, 0]))
m_π = ForwardDiff.hessian(fid_π, [0.0, 0.0])
display(m_π)
calib_scale = m_π[1, 1];

1.0

2×2 Matrix{Float64}:
 -4.9348        3.70051e-32
  3.70051e-32  -4.9348

## $\pi$ pulse calibration
The $\pi$ pulse population infidelity for the two chambers are measured to be $0.5%$ and $1%$.

In [3]:
err_a::Float64 = 0.5e-2;
err_b::Float64 = 3.6e-2;

In [4]:
function disp_sys(sys, cb, name)
    @assert all(cb(sys, [0, 0]) .≈ (1, 1))
    mp = ForwardDiff.hessian((x)->cb(sys, x)[1], [0.0, 0.0])
    mc = ForwardDiff.hessian((x)->cb(sys, x)[2], [0.0, 0.0])
    err_p = mp[1, 1] * (err_a + err_b) / calib_scale * 100
    err_c = mc[1, 1] * (err_a + err_b) / calib_scale * 100
    display(Markdown.parse("## Error for $(name)"))
    display(Markdown.parse("**Population: $(round(err_p, digits=2))%**"))
    display(Markdown.parse("**Parity: $(round(err_c, digits=2))%**"))
    display(Markdown.parse("**Total: $(round((err_p + err_c) / 2, digits=2))%**"))
    display(mp)
    display(mc)
    display((mp .+ mc) ./ 2)
    return
end

disp_sys (generic function with 1 method)

In [5]:
struct EmitWrapper{Sys,Ta,Tb,P}
    sys::Sys
    pa::Ta
    pb::Tb
    projs::Vector{P}
end
function EmitWrapper(sys, pa, pb)
    proja1 = sys.proj1s[1] * sqrt(pa)
    proja0 = sys.proj1s[1] * sqrt(1 - pa) + sys.proj1comps[1]
    projb1 = sys.proj1s[1] * sqrt(pb)
    projb0 = sys.proj1s[1] * sqrt(1 - pb) + sys.proj1comps[1]
    projs = [proja0 ⊗ projb0, proja1 ⊗ projb0 + proja0 ⊗ projb1, proja1 ⊗ projb1]
    return EmitWrapper(sys, pa, pb, projs)
end
function detect_success(ρ, ew::EmitWrapper)
    return 2 * apply(ρ, ew.projs[3]) + apply(ρ, ew.projs[2])
end
function detect_fail(ρ, ew::EmitWrapper)
    return apply(ρ, ew.projs[3]) + apply(ρ, ew.projs[2]) + apply(ρ, ew.projs[1])
end

detect_fail (generic function with 1 method)

In [6]:
pa::Float64 = 0.7;
pb::Float64 = 0.7;

In [7]:
function init_state(sw::SysWrapper{Sys} where Sys<:System{2}, ew::EmitWrapper, success)
    ψ = R2_ratio(sw, 1, 2, 1 / 2) * sw.sys.ψ2s[1]
    ρ = projector(ψ)
    ρ = 1 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    return 2 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
end

function fid01_10(sys::System{2}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 2))
    # 01 -> 10
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end

fid01_10 (generic function with 1 method)

# $d=2$

In [8]:
display(md"# Excitation probability **\$p=0.7\$**")
pa::Float64 = 0.7;
pb::Float64 = 0.7;
disp_sys(sys2, fid01_10, "01 -> 10")
display(md"# Excitation probability **\$p=1.0\$**")
pa::Float64 = 1;
pb::Float64 = 1;
disp_sys(sys2, fid01_10, "01 -> 10")

# Excitation probability **$p=0.7$**


## Error for 01 -> 10


**Population: 5.33%**


**Parity: 11.87%**


**Total: 8.6%**


...

**Parity: 6.15%**


**Total: 5.12%**


2×2 Matrix{Float64}:
 -4.9348  -4.9348
 -4.9348  -4.9348

2×2 Matrix{Float64}:
 -7.4022       -1.33227e-15
 -1.77636e-15  -7.4022

2×2 Matrix{Float64}:
 -6.1685  -2.4674
 -2.4674  -6.1685

In [9]:
function init_state(sw::SysWrapper{Sys} where Sys<:System{3}, ew::EmitWrapper, success)
    ψ = R2_ratio(sw, 1, 3, 1 / 2) * R2_ratio(sw, 1, 2, 1 / 3) * sw.sys.ψ2s[1]
    ρ = projector(ψ)
    ρ = 1 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    return 3 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
end

function fid01_12(sys::System{3}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 2))
    # 01 -> 12
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end
function fid02_10(sys::System{3}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 3))
    # 02 -> 10
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end
function fid12_20(sys::System{3}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (2, 3))
    # 12 -> 20
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 3])) + tr(apply(ρ, sys.proj2s[3, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[3, 3])))
    return P_odd, c
end

fid12_20 (generic function with 1 method)

# $d=3$:

In [10]:
display(md"# Excitation probability **\$p=0.7\$**")
pa::Float64 = 0.7;
pb::Float64 = 0.7;
disp_sys(sys3, fid01_12, "01 -> 12")
disp_sys(sys3, fid02_10, "02 -> 10")
disp_sys(sys3, fid12_20, "12 -> 20")
display(md"# Excitation probability **\$p=1.0\$**")
pa::Float64 = 1;
pb::Float64 = 1;
disp_sys(sys3, fid01_12, "01 -> 12")
disp_sys(sys3, fid02_10, "02 -> 10")
disp_sys(sys3, fid12_20, "12 -> 20")

# Excitation probability **$p=0.7$**


## Error for 01 -> 12


**Population: 5.33%**


**Parity: 8.92%**


**Total: 7.12%**


...

**Parity: 8.31%**


**Total: 7.23%**


2×2 Matrix{Float64}:
 -7.4022  -4.9348
 -4.9348  -7.4022

2×2 Matrix{Float64}:
 -10.0049     0.13532
   0.13532  -10.0049

2×2 Matrix{Float64}:
 -8.70356  -2.39974
 -2.39974  -8.70356

In [11]:
function init_state(sw::SysWrapper{Sys} where Sys<:System{4}, ew::EmitWrapper, success)
    ψ = R2_ratio(sw, 1, 4, 1 / 2) * R2_ratio(sw, 1, 3, 1 / 3) * R2_ratio(sw, 1, 2, 1 / 4) * sw.sys.ψ2s[1]
    ρ = projector(ψ)
    ρ = 1 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = 3 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    return 4 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
end
function fid01_12(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 2))
    # 01 -> 12
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end
function fid02_13(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 3))
    # 02 -> 13
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end
function fid03_10(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 4))
    # 03 -> 10
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end
function fid12_23(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (2, 3))
    # 12 -> 23
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 3])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 3])) + tr(apply(ρ, sys.proj2s[3, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[3, 3])))
    return P_odd, c
end
function fid13_20(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (2, 4))
    # 13 -> 20
    ρ = apply(ρ, sys.proj2sub2[1, 3])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 3])) + tr(apply(ρ, sys.proj2s[3, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[3, 3])))
    return P_odd, c
end
function fid23_30(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (3, 4))
    # 23 -> 30
    ρ = apply(ρ, sys.proj2sub2[1, 4])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 4])) + tr(apply(ρ, sys.proj2s[4, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[4, 4])))
    return P_odd, c
end

fid23_30 (generic function with 1 method)

# $d=4$:

In [12]:
display(md"# Excitation probability **\$p=0.7\$**")
pa::Float64 = 0.7;
pb::Float64 = 0.7;
disp_sys(sys4, fid01_12, "01 -> 12")
disp_sys(sys4, fid02_13, "02 -> 13")
disp_sys(sys4, fid03_10, "03 -> 10")
disp_sys(sys4, fid12_23, "12 -> 23")
disp_sys(sys4, fid13_20, "13 -> 20")
disp_sys(sys4, fid23_30, "23 -> 30")
display(md"# Excitation probability **\$p=1.0\$**")
pa::Float64 = 1;
pb::Float64 = 1;
disp_sys(sys4, fid01_12, "01 -> 12")
disp_sys(sys4, fid02_13, "02 -> 13")
disp_sys(sys4, fid03_10, "03 -> 10")
disp_sys(sys4, fid12_23, "12 -> 23")
disp_sys(sys4, fid13_20, "13 -> 20")
disp_sys(sys4, fid23_30, "23 -> 30")

# Excitation probability **$p=0.7$**


## Error for 01 -> 12


**Population: 5.33%**


**Parity: 10.04%**


**Total: 7.68%**


...

**Parity: 6.26%**


**Total: 5.18%**


2×2 Matrix{Float64}:
 -4.9348  -4.9348
 -4.9348  -4.9348

2×2 Matrix{Float64}:
 -7.53752   0.13532
  0.13532  -7.53752

2×2 Matrix{Float64}:
 -6.23616  -2.39974
 -2.39974  -6.23616